In [1]:
!pip3 install h5py

In [48]:
import h5py
import numpy as np

def openFile(filename):
    with h5py.File(filename, 'r') as f:
        print("Keys: %s" % f.keys())
        a_group_key = list(f.keys())[0]
        data = list(f[a_group_key])
        axis0_data = list(f[a_group_key]['axis0'])
        axis1_data = list(f[a_group_key]['axis1'])
        block0_items = list(f[a_group_key]['block0_items'])
        block0_values = list(f[a_group_key]['block0_values'])
        separate_arrays = [np.array(array) for array in block0_values]
        
        return {
            'data': data,
            'axis0_data': axis0_data,
            'axis1_data': axis1_data,
            'block0_items': block0_items,
            'block0_values': separate_arrays
        }
    
    
data1 = openFile('Data/uni_111.h5')
data2 = openFile('Data/uni_112.h5')
data3 = openFile('Data/uni_113.h5')

Keys: <KeysViewHDF5 ['EEG']>
Keys: <KeysViewHDF5 ['EEG']>
Keys: <KeysViewHDF5 ['EEG']>


In [59]:
def dataAnalysis(data):
    print("There are %d electrode planes from which data is collected" % len(data['axis0_data']))
    print("With frequency of 200 signals in a seconnd, the data is collected in %.2f minutes or %d seconds" % (len(data['axis1_data'])/60/200, len(data['axis1_data'])/200))
    print("-----------------------------------------------------")

dataAnalysis(data1)
dataAnalysis(data2)
dataAnalysis(data3)
       

There are 44 electrode planes from which data is collected
With frequency of 200 signals in a seconnd, the data is collected in 7.90 minutes or 474 seconds
-----------------------------------------------------
There are 44 electrode planes from which data is collected
With frequency of 200 signals in a seconnd, the data is collected in 8.43 minutes or 506 seconds
-----------------------------------------------------
There are 44 electrode planes from which data is collected
With frequency of 200 signals in a seconnd, the data is collected in 9.13 minutes or 548 seconds
-----------------------------------------------------


In [49]:
print(len(data3['axis1_data']))

109600
